In [16]:
%reset -f
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from utils import init_location, cal_z_locid, cal_velz_loc, \
                  interpolate_vel, data_normalize, To_Tensor
# from parflow.tools.hydrology import calculate_water_table_depth, \
# calculate_overland_flow_grid, calculate_overland_fluxes
# %matplotlib widget

In [17]:
p_start = 35040
t_start = 0 #iteration start
t_end = 10 #iteration end
real_path = 'True'
selectd = np.array([12,15,2,4,12,15])

npcell = 5  #used for train
testsize = 1  #used for test
npcell = npcell + testsize  #all
# how many particles in a grid cell

interval = 1 #always no gap
step_size = 1 #always one hour

In [18]:
# input model parameters
nx = 20
ny = 5
nz = 20

dx = 5
dy = 0.2
dz = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, \
               0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.1])

boundings = np.array([0., 100., 0., 1., 0., 9.4])

In [19]:
for ii in range(t_start,t_end):
    
    istep = 1 + ii #current iterate step
    print('istep',istep)

    Vx = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velx.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vy = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.vely.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Vz = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.velz.' \
                                    +str(istep+p_start).zfill(5)+'.pfb'))
    Saturation = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.satur.' \
                                            +str(istep+p_start).zfill(5)+'.pfb'))
    Porosity = read_pfb(get_absolute_path('./vel_field/hillslope_clm_ER_shrub.out.porosity.pfb'))
    # print(Vx.shape, Vy.shape, Vz.shape)

    if real_path == 'False' or (real_path == 'True' and istep == (t_start + 1)):
        
        #setup initial location
        initx,inity,initz = init_location(npcell,nx,ny,nz,dx,dy,dz)
        # initz[1,0,:,:]

        #cal locid
        Px = np.copy(initx)
        Py = np.copy(inity)
        Pz = np.copy(initz)
        
        print('real_path',real_path)
    
    if real_path == 'True' and istep != (t_start + 1):
        
        initx = np.copy(Px)
        inity = np.copy(Py)
        initz = np.copy(Pz)
        
        print('real_path',real_path)

    Plocx = np.floor(Px/dx)
    Plocy = np.floor(Py/dy)
    Plocz = cal_z_locid(npcell,nx,ny,nz,dz,Pz)

    # cal vel loc
    # this can be got by random number directly
    # but for multiple steps, we still need to calculate
    # so we provide the template here
    # we may also calculate it from random number
    # i mean like add the intial fraction with the new_distance/vel
    # but additional other calculation may need

    Clocx = (Px - Plocx*dx)/dx
    Clocy = (Py - Plocy*dy)/dy
    Clocz = cal_velz_loc(npcell,nx,ny,nz,dz,Pz,Plocz)

    Vpx, Vpy, Vpz = interpolate_vel(npcell,nx,ny,nz,Plocx,Plocy,Plocz,Clocx,Clocy,Clocz, \
                                   Vx,Vy,Vz,Saturation,Porosity)

    Px[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Px[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpx[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]
    
    Py[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Py[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpy[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]
    
    Pz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] = \
    Pz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]] + \
    Vpz[:,selectd[0]:selectd[1],selectd[2]:selectd[3],selectd[4]:selectd[5]]

    tt,initx1,inity1,initz1,Px1,Py1,Pz1 = data_normalize(istep,interval,step_size,t_start,t_end, \
                                                   initx,inity,initz,Px,Py,Pz,boundings)

    start_train_new, end_train_new, time_train_new = To_Tensor(0,npcell-testsize, \
               selectd[0],selectd[2],selectd[4],selectd[1],selectd[3],selectd[5], \
                                                initx1,inity1,initz1,Px1,Py1,Pz1,tt)
    start_test_new, end_test_new, time_test_new = To_Tensor(npcell-testsize,npcell, \
               selectd[0],selectd[2],selectd[4],selectd[1],selectd[3],selectd[5], \
                                                initx1,inity1,initz1,Px1,Py1,Pz1,tt)
    
    if istep == t_start + 1:
        start_train = start_train_new
        end_train   = end_train_new
        time_train  = time_train_new
        
        start_test  = start_test_new
        end_test    = end_test_new
        time_test   = time_test_new
        
    else:
        
        start_train = torch.cat((start_train, start_train_new),dim=0)
        end_train   = torch.cat((end_train, end_train_new),dim=0)
        time_train  = torch.cat((time_train, time_train_new),dim=0)
        
        start_test  = torch.cat((start_test, start_test_new),dim=0)
        end_test    = torch.cat((end_test, end_test_new),dim=0)
        time_test   = torch.cat((time_test, time_test_new),dim=0)
    

istep 1
real_path True
istep 2
real_path True
istep 3
real_path True
istep 4
real_path True
istep 5
real_path True
istep 6
real_path True
istep 7
real_path True
istep 8
real_path True
istep 9
real_path True
istep 10
real_path True


In [20]:
print(start_train.shape,start_test.shape,end_train.shape,end_test.shape)

torch.Size([900, 3]) torch.Size([180, 3]) torch.Size([900, 3]) torch.Size([180, 3])


In [21]:
data_train = TensorDataset(start_train,end_train,time_train)
data_test = TensorDataset(start_test,end_test,time_test)

if real_path == 'True':
    torch.save(data_train,'data_check.pth')  
else:
    torch.save(data_train,'data_train.pth')
    torch.save(data_test,'data_test.pth')

In [24]:
count = 0
for x in data_train:#generator
    print(x)
    count = count + 1
    if count > 1000: break

(tensor([ 0.2965, -0.0903,  0.3790]), tensor([ 0.2965, -0.0903,  0.3789]), tensor([-0.8000]))
(tensor([ 0.3052, -0.1446,  0.3593]), tensor([ 0.3052, -0.1446,  0.3592]), tensor([-0.8000]))
(tensor([ 0.4853, -0.1415,  0.3478]), tensor([ 0.4853, -0.1415,  0.3477]), tensor([-0.8000]))
(tensor([0.2500, 0.2048, 0.3809]), tensor([0.2500, 0.2048, 0.3808]), tensor([-0.8000]))
(tensor([0.3412, 0.2997, 0.3665]), tensor([0.3412, 0.2997, 0.3664]), tensor([-0.8000]))
(tensor([0.4171, 0.5097, 0.3809]), tensor([0.4171, 0.5097, 0.3808]), tensor([-0.8000]))
(tensor([ 0.2557, -0.1082,  0.3968]), tensor([ 0.2557, -0.1082,  0.3966]), tensor([-0.8000]))
(tensor([0.3000, 0.1226, 0.4128]), tensor([0.3000, 0.1226, 0.4127]), tensor([-0.8000]))
(tensor([0.4478, 0.1239, 0.4084]), tensor([0.4478, 0.1239, 0.4082]), tensor([-0.8000]))
(tensor([0.2104, 0.3011, 0.4279]), tensor([0.2104, 0.3011, 0.4278]), tensor([-0.8000]))
(tensor([0.3441, 0.2288, 0.4096]), tensor([0.3441, 0.2288, 0.4094]), tensor([-0.8000]))
(tensor(